### Imports

In [15]:
import os
import json
import datetime
import random
from openai.types.chat import ChatCompletionMessage, ChatCompletionMessageToolCall
from openai import AzureOpenAI
from tenacity import retry, wait_random_exponential, stop_after_attempt
from typing import List


### External Services Simulation
This could be replaced with any backend, api, integration service, etc...

In [16]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "bogota" in location.lower():
        return json.dumps({"location": "Bogota", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    
def get_n_day_weather_forecast(location, unit="fahrenheit", num_days=3):
    current_date = datetime.date.today()
    forecast = []
    
    for i in range(num_days):
        date = current_date + datetime.timedelta(days=i)
        temperature = random.randint(10, 20)
        forecast.append(f"{date.strftime('%B %d')}: {temperature}")
    
    return json.dumps({"location": location, "forecast": forecast, "unit": unit})

### Helper function to visualize responses from LLM about what function and parameters should be called

In [17]:
def pretty_print_chat_completion_message(chat_message: ChatCompletionMessage):
    content = chat_message.content
    print("Content:")
    print(json.dumps(content, indent=2))
    
    if chat_message.tool_calls:
        for tool_call in chat_message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments
            print("Function Name:", function_name)
            print("Arguments:")
            print(json.dumps(json.loads(arguments), indent=2))
    else:
        print("Functions: None")

### Azure OpenAI client setup

In [18]:
client = AzureOpenAI(
    api_key=os.getenv("AI_ROADSHOW_AOAI_KEY"),  
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AI_ROADSHOW_AOIA_ENDPOINT")
)
deployment = "ai-roadshow" #gpt-3.5-turbo-0613
messages = []

### Helper function to actually make the call to the function suggested by the LLM

In [19]:
def function_caller(tool_calls: List[ChatCompletionMessageToolCall]):

    if tool_calls:
        available_functions = {
            "get_current_weather": get_current_weather,
            "get_n_day_weather_forecast": get_n_day_weather_forecast,	
        }  
        
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            if function_name == "get_current_weather":
                function_response = function_to_call(
                    location=function_args.get("location"),
                    unit=function_args.get("unit"))
            elif function_name == "get_n_day_weather_forecast":
                function_response = function_to_call(
                    location=function_args.get("location"),
                    unit=function_args.get("unit"),
                    num_days=function_args.get("num_days"))
            
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }                
            )           

    response = client.chat.completions.create(
        model=deployment,
        messages=messages,
    ).choices[0].message.content # get a new response from the model where it can see the function response
    
    return response

### Model interaction method with retry policy

In [20]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=deployment):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

### List of functions available for our aplication

In [21]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city for which to get the weather",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["fahrenheit", "celsius"],
                        "description": "The temperature unit to use. Default is fahrenheit.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location the user wants to know its weather",
                    },
                    "format": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Default is fahrenheit.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

### Testing specifying location

In [22]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "I'm visiting San Francisco. What will be the weather today?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
pretty_print_chat_completion_message(assistant_message)
print(function_caller(assistant_message.tool_calls))

Content:
null
Function Name: get_current_weather
Arguments:
{
  "location": "San Francisco",
  "format": "fahrenheit"
}
The current temperature in San Francisco is 72 degrees Fahrenheit.


### Testing specifying the location and the preferred format

In [23]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Please tell me the weather in Bogota, in celsius"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
pretty_print_chat_completion_message(assistant_message)
print(function_caller(assistant_message.tool_calls))

Content:
null
Function Name: get_current_weather
Arguments:
{
  "location": "Bogota",
  "format": "celsius"
}
The current temperature in Bogota is 10 degrees Celsius.


### Testing without specifying location

In [24]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "How's going to be the weather?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
pretty_print_chat_completion_message(assistant_message)
print(function_caller(assistant_message.tool_calls))

Content:
"Sure! Could you please specify the location for which you want to know the weather?"
Functions: None
I'm sorry, but I need the specific location for which you want the weather forecast. Can you please provide the city or town name?


#### User answers the ask for location specification

In [25]:
messages.append({"role": "user", "content": "I am in Paris"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message
pretty_print_chat_completion_message(assistant_message)
print(function_caller(assistant_message.tool_calls))


Content:
null
Function Name: get_current_weather
Arguments:
{
  "location": "Paris",
  "format": "celsius"
}
The current temperature in Paris is 22 degrees Celsius.


### Testing the other function

In [26]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "How will be the weather in Cartagena for the next week?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
pretty_print_chat_completion_message(assistant_message)
print(function_caller(assistant_message.tool_calls))


Content:
null
Function Name: get_n_day_weather_forecast
Arguments:
{
  "location": "Cartagena",
  "format": "celsius",
  "num_days": 7
}
The weather in Cartagena for the next week will be as follows:
- March 19: 13°C
- March 20: 12°C
- March 21: 10°C
- March 22: 16°C
- March 23: 11°C
- March 24: 19°C
- March 25: 18°C


### Parallel Function Calling
Newer models like gpt-4-1106-preview or gpt-3.5-turbo-1106 can call multiple functions in one turn.

In [27]:
client = AzureOpenAI(
    api_key=os.getenv("AI_ROADSHOW_AOAI_1106_KEY"),  
    api_version="2024-02-01",
    azure_endpoint=os.getenv("AI_ROADSHOW_AOIA_1106_ENDPOINT")
)

deployment = "ai-roadshow-1106" #gpt-3.5-turbo-1106 is available in WEST US among selected regions
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in San Francisco and Bogota over the next 4 days"})
chat_response = chat_completion_request(
    messages, tools=tools, model=deployment
)

assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
pretty_print_chat_completion_message(assistant_message)
print(function_caller(assistant_message.tool_calls))

Content:
null
Function Name: get_n_day_weather_forecast
Arguments:
{
  "location": "San Francisco",
  "format": "fahrenheit",
  "num_days": 4
}
Function Name: get_n_day_weather_forecast
Arguments:
{
  "location": "Bogota",
  "format": "celsius",
  "num_days": 4
}
The weather forecast for the next 4 days is as follows:
- San Francisco: March 19: 20°F, March 20: 13°F, March 21: 15°F, March 22: 11°F
- Bogota: March 19: 15°C, March 20: 15°C, March 21: 19°C, March 22: 10°C
